In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [2]:
def get_spielplan():
    URL = "https://dsvdaten.dsv.de/Modules/WB/League.aspx?Season=2024&LeagueID=77&Group=&LeagueKind=L&StateID=17"
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, "html.parser")

    spielplan = soup.find_all("table")[1]
    results = []

    for spiele in spielplan.find_all("tr")[2:]:
        cols = spiele.find_all("td")

        ergebnis_tag = cols[5].find("a")
        protokoll_link = ergebnis_tag.get("href", "") if ergebnis_tag else ""

        results.append({
            "Spielnummer": cols[0].text,
            "Datum & Uhrzeit": cols[1].text,
            "Heim": cols[2].text,
            "Gast": cols[3].text,
            "Ort": cols[4].text,
            "Ergebnis": cols[5].text,
            "Viertel": cols[6].text,
            "Protokoll": protokoll_link
        })

    return pd.DataFrame(results)

def cleanse_text_fields(df):
    def clean_cell(x):
        if not isinstance(x, str):
            return x
        return (
            x.strip()                              # Leerzeichen vorn/hinten
             .replace("\u00a0", " ")                # geschütztes Leerzeichen
             .replace("\n", " ")                    # Zeilenumbrüche entfernen
             .replace("\r", "")                     # Wagenrücklauf entfernen
             .lower()                               # Kleinbuchstaben
             .encode("ascii", "ignore").decode()    # Akzente/Sonderzeichen entfernen
        )

    return df.applymap(clean_cell)

def expand_input_fields(df):
    # Viertelergebnisse splitten
    viertel_liste = df["Viertel"].apply(lambda x: re.findall(r"(\d+):(\d+)", x))

    for i in range(5):
        df[f"Q{i+1}_Heim"] = viertel_liste.apply(lambda x: x[i][0] if len(x) > i else None).astype("Int64")
        df[f"Q{i+1}_Gast"] = viertel_liste.apply(lambda x: x[i][1] if len(x) > i else None).astype("Int64")

    #Heim:Gast teilen
    tore = df["Ergebnis"].str.extract(r"(?P<Heim_Tore>\d+)\s*[:]\s*(?P<Gast_Tore>\d+)")

    df["Heim_Tore"] = pd.to_numeric(tore["Heim_Tore"], errors="coerce").astype("Int64")
    df["Gast_Tore"] = pd.to_numeric(tore["Gast_Tore"], errors="coerce").astype("Int64")

    #Datum teilen
    df[["Datum_str", "Uhrzeit_str"]] = df["Datum & Uhrzeit"].str.split(",", n=1, expand=True)
    df["Datum_str"] = df["Datum_str"].str.strip()
    df["Uhrzeit_str"] = df["Uhrzeit_str"].str.strip().str.replace(" uhr", "", case=False)
    df["Datum"] = pd.to_datetime(df["Datum_str"], format="%d.%m.%y", errors="coerce")
    df["Uhrzeit"] = pd.to_datetime(df["Uhrzeit_str"], format="%H:%M", errors="coerce").dt.time
    def combine_date_time(row):
        if pd.notnull(row["Datum"]) and pd.notnull(row["Uhrzeit"]):
            return pd.to_datetime(f"{row['Datum']} {row['Uhrzeit']}")
        else:
            return pd.NaT
    df["Datum_Uhrzeit"] = df.apply(combine_date_time, axis=1)

    #Weitere Datumsfelder berechnen
    df["Wochentag"] = None
    df["Wochenende"] = None
    df["Spieltagstyp"] = None

    if "Datum" in df.columns:
        # Nur gültige Datumswerte verwenden
        valid_dates = df["Datum"].notna()

        df.loc[valid_dates, "Wochentag"] = df.loc[valid_dates, "Datum"].dt.day_name(locale="de_DE")
        df.loc[valid_dates, "Wochenende"] = df.loc[valid_dates, "Datum"].dt.weekday >= 5
        df.loc[valid_dates, "Spieltagstyp"] = df.loc[valid_dates, "Wochenende"].apply(
            lambda x: "Wochenende" if x else "Wochentag"
        )
    
    return df

In [3]:
#spielplan = get_spielplan()
#spielplan = cleanse_text_fields(spielplan)
#spielplan = expand_input_fields(spielplan)

In [37]:
def get_tabelle():
    URL = "https://dsvdaten.dsv.de/Modules/WB/League.aspx?Season=2024&LeagueID=77&Group=&LeagueKind=L&StateID=17"
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, "html.parser")

    tabelle = soup.find_all("table")[2]

    results = []

    for team in tabelle.find_all("tr")[1:]:
        cols = team.find_all("td")

        if len(cols) < 9:
            continue

        results.append({
            "Platzierung": cols[0].text,
            "Team": cols[1].text,
            "Spiele": cols[2].text,
            "Siege": cols[3].text,
            "Unentschieden": cols[4].text,
            "Niederlagen": cols[5].text,
            "Tore": cols[6].text,
            "Tordifferenz": cols[7].text,
            "Punkte": cols[8].text
        })

    return pd.DataFrame(results)

In [38]:
get_tabelle()

,Platzierung,Team,Spiele,Siege,Unentschieden,Niederlagen,Tore,Tordifferenz,Punkte
0,1,SV Blau-Weiß Bochum,8/16,8,0,0,138:80,+58,24
1,2,SGW Rhenania/BW Poseidon Köln,9/16,7,0,2,141:90,+51,20
2,3,SV Krefeld 1972 II,9/16,6,0,3,113:103,+10,18
3,4,SGW SC Solingen/Wfr. Wuppertal,9/16,5,0,4,89:103,-14,15
4,5,Duisburger SV 1898 II,6/16,4,0,2,72:57,+15,12
5,6,ASC Duisburg II,8/16,3,0,5,118:109,+9,10
6,7,SV Lünen 08 - dir. Vergleich: 1. (Punkte: 3),9/16,2,0,7,108:128,-20,6
7,8,SG Wasserball Essen - dir. Vergleich: 2. (Punk...,9/16,2,0,7,88:127,-39,6
8,9,Aachener SV 06,9/16,1,0,8,59:129,-70,3
